# Transfermarkt Scraper

In [8]:
from urllib.request import *
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import scipy.stats as stats
import pylab as pl
import requests
import json
import os
%matplotlib inline

In [9]:
base_url = "https://www.transfermarkt.com"

In [45]:
def parsePlayer(player_ref):
    transferList = []
    
    params = {
    }

    HEADERS = {'User-Agent': 'Mozilla/5.0'}
    url = base_url + "/" + href

    r = requests.get(url, headers=HEADERS)
    response = BeautifulSoup(r.text, 'html.parser')
    return response
    '''
    html_reviews = response.find_all("div", {"class": "review-container clearfix"})
    reviews = []
    
    for article in articles:
        if article.h3 is not None:
            name = removeSpace(article.h3.text)
            if name != "":
                print(name)
                try:
                    ratingStars = article.find("div",{"class" : "rating-stars"})["data-ratingstars"]
                    nbReviews = article.find("format-large-number")["number"]
                except:
                    ratingStars = None
                    nbReviews = None
                link = article.find('a', href=re.compile('^/recipe/'))
                if link is not None:
                    print(link['href'])
                    articlesList.append([name, ratingStars, nbReviews, link['href']])
                else:
                    link = article.find('a', href=re.compile('^/cook/'))
                    if link is not None:
                        print(link['href'])
                        articlesList.append([name, ratingStars, nbReviews, link['href']])
                    else:
                        print("REALLY NO LINKS")
    return articlesList
    '''

In [46]:
href = "jordi-alba/profil/spieler/69751"
parsePlayer(href)


<!DOCTYPE html>

<!-- paulirish.com/2008/conditional-stylesheets-vs-css-hacks-answer-neither/ -->
<!--[if IE 7]>    <html class="ie7 oldie" lang="en"> <![endif]-->
<!--[if IE 8]>    <html class="no-js lt-ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="x-ua-compatible"/>
<meta content="telephone=no" name="format-detection"/>
<meta content="#1a3151" name="theme-color">
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/favicon-16x16.png" rel="shortcut icon" sizes="16x16"/>
<link href="/android-chrome-192x192.png" rel="shortcut icon" sizes="192x192"/>
<link href="/apple-touch-icon-152x152.png" rel="apple-touch-icon-precomposed"/>
<link href="http://www.transfermarkt.de/jordi-alba/profil/spieler/69751" hreflang="x-default" rel="alternate">
<link href="http://www.transfermarkt.de/jordi-alba/profil/spieler/69751" hreflang="de" rel="alternate">
